In [ ]:
%matplotlib inline

import os

#if using Theano with GPU
#os.environ["KERAS_BACKEND"] = "tensorflow"

import random
import numpy as np
import keras

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

import keras.utils as image
from keras.applications.imagenet_utils import preprocess_input
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Model

from google.colab import files

In [ ]:
# Coletando o Dataset

! pip install -q kaggle

files.upload() # Upload do Token API do Kaggle

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list

In [ ]:
! kaggle datasets download -d chrisfilo/fruit-recognition

In [ ]:
! ls

In [ ]:
! mkdir train
! unzip fruit-recognition.zip -d train

In [ ]:
!mkdir Train
!mv train/Plum/ Train/

In [ ]:
!mv train/Tomatoes Train/

In [ ]:
!ls Train/Tomatoes # Vendo se está tudo correto

In [ ]:
# Pegando uma rede neural pré treinada (InceptionV3)

pre_model = keras.applications.InceptionV3(weights='imagenet', include_top=True)
pre_model.summary()

In [ ]:
num_classes = 2 # Tomates e ameixas

inp = pre_model.input
out_layer = Dense(num_classes, activation='softmax')
out = out_layer(pre_model.layers[-2].output)
new_model = Model(inp, out)

In [ ]:
for l, layer in enumerate(new_model.layers[:-1]):
  print(f'1 - {l}; {layer}')
  layer.trainable = False

for l, layer in enumerate(new_model.layers[-1:]):
  print(f'2 - {l}; {layer}')
  layer.trainable = True

In [ ]:
new_model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

new_model.summary()

In [ ]:
# Pré-processamento das imagens

root = 'Train'
train_split, val_split = 0.7, 0.15

categories = [x[0] for x in os.walk(root) if x[0]][1:]

def get_image(path):
    img = image.load_img(path, target_size=(299, 299))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return img, x

data = []
for c, category in enumerate(categories):
    images = [os.path.join(dp, f) for dp, dn, filenames 
              in os.walk(category) for f in filenames 
              if os.path.splitext(f)[1].lower() in ['.jpg','.png','.jpeg']]
    for img_path in images:
        img, x = get_image(img_path)
        data.append({'x':np.array(x[0]), 'y':c})

In [ ]:
random.shuffle(data)

In [ ]:
idx_val = int(train_split * len(data))
idx_test = int((train_split + val_split) * len(data))
train = data[:idx_val]
val = data[idx_val:idx_test]
test = data[idx_test:]

In [ ]:
x_train, y_train = np.array([t["x"] for t in train]), [t["y"] for t in train]
x_val, y_val = np.array([t["x"] for t in val]), [t["y"] for t in val]
x_test, y_test = np.array([t["x"] for t in test]), [t["y"] for t in test]

In [ ]:
# Liberando memória
data.clear()
train.clear()
val.clear()
test.clear()

In [ ]:
# Normalizando o dados
x_train = x_train.astype('float32') / 255.
x_val = x_val.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

y_train.shape

In [ ]:
# Exemplo de imagens

images = [os.path.join(dp, f) for dp, dn, filenames in os.walk(root) for f in filenames if os.path.splitext(f)[1].lower() in ['.jpg','.png','.jpeg']]
idx = [int(len(images) * random.random()) for i in range(8)]
imgs = [image.load_img(images[i], target_size=(299, 299)) for i in idx]
concat_image = np.concatenate([np.asarray(img) for img in imgs], axis=1)
plt.figure(figsize=(16,4))
plt.imshow(concat_image)

In [ ]:
# Treinando o modelo

history = new_model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=10,
                    validation_data=(x_val, y_val))

In [ ]:
plt.imshow(x_test[4])
pred = np.argmax(new_model.predict(np.expand_dims(x_test[4], axis=0)))

if pred == 0: print('O modelo acha que é um tomate')
if pred == 1: print('O modelo acha que é uma ameixa')
print(y_test[4])

In [ ]:
loss, accuracy = new_model.evaluate(x_test, y_test, verbose=0)

In [ ]:
accuracy # Acertou 99,4% das vezes

In [ ]:
loss